<a href="https://colab.research.google.com/github/Berchie-Sam/Glaucoma_Detection/blob/main/%5Bfinal_project%5DGlaucomaDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


#**Importing libraries and packages**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import random
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, Lambda, Reshape, AveragePooling2D, Dense, ReLU, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from os import listdir
from matplotlib import image
from keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.layers import Layer
from keras import activations
from keras import utils
import joblib
import math
from multiprocessing import Pool
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
!pip install visualkeras
import visualkeras
from tensorflow.keras.preprocessing import image
%matplotlib inline
from sklearn.model_selection import KFold
from keras import initializers
from google.colab import drive
from tensorflow.keras.models import load_model
import seaborn as sns
from PIL import Image

# 1. **Loading Dataset**

In [ ]:
print(tf.__version__)



> a) Accessing TPU runtime



In [ ]:

# Check if the TPU system has already been initialized
if 'COLAB_TPU_ADDR' in os.environ:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
else:
    strategy = tf.distribute.OneDeviceStrategy('GPU:0')  # Use GPU if TPU is not available


In [ ]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/Projects/Final(Undergraduate)/[cleaned]RIM_ONE'

#**EXPLORATORY DATA ANALYSIS**

In [ ]:
# get the list of png files from sub image class folders
with strategy.scope():
  # get the list of png files from sub image class folders
  normal_imgs = [fn for fn in os.listdir(os.path.join(data_dir, 'Normal')) if (fn.endswith('.png'))]
  glaucoma_imgs = [fn for fn in os.listdir(os.path.join(data_dir, 'Glaucoma')) if (fn.endswith('.png'))]

  # Check if the lists are empty
  if len(normal_imgs) == 0 or len(glaucoma_imgs) == 0:
      raise ValueError("No images found in the specified directories.")


In [ ]:
# Calculate the total number of images
total_images = len(normal_imgs) + len(glaucoma_imgs)

# Print the total number of images
print("Total number of images:", total_images)

# Print the number of images in each class
print("Number of Normal images:", len(normal_imgs))
print("Number of Glaucoma images:", len(glaucoma_imgs))
print('')

# Define class labels and counts
class_labels = ['Normal', 'Glaucoma']
class_counts = [len(normal_imgs), len(glaucoma_imgs)]

# Create a bar plot for the image distribution
plt.bar(class_labels, class_counts)
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Image Distribution by Class')
plt.show()

In [ ]:
from PIL import Image

# randomly select 3 of each
select_norm = np.random.choice(normal_imgs, 3, replace=False)
select_glau = np.random.choice(glaucoma_imgs, 3, replace=False)

# plotting 2 x 3 image matrix
fig = plt.figure(figsize=(8, 6))
for i in range(6):
    if i < 3:
        fp = os.path.join(data_dir, 'Normal', select_norm[i])
        label = 'Normal'
    else:
        fp = os.path.join(data_dir, 'Glaucoma', select_glau[i-3])
        label = 'Glaucoma'
    ax = fig.add_subplot(2, 3, i + 1)

    # Load the image using PIL (Pillow) without converting to grayscale
    fn = Image.open(fp)
    plt.imshow(fn)  # Display the image in its original color
    plt.title(label)
    plt.axis('off')
plt.show()


In [ ]:
# also check the number of files here
len(normal_imgs), len(glaucoma_imgs)

def img2np(path, list_of_filename, size=(64, 64)):
    # iterating through each file
    for fn in list_of_filename:
        fp = os.path.join(path, fn)
        current_image = Image.open(fp).convert("L")  # Load as grayscale
        current_image = current_image.resize(size)
        # covert image to a matrix
        img_ts = np.array(current_image)
        # turn that into a vector / 1D array
        img_ts = [img_ts.ravel()]
        try:
            # concatenate different images
            full_mat = np.concatenate((full_mat, img_ts))
        except UnboundLocalError:
            # if not assigned yet, assign one
            full_mat = img_ts
    return full_mat

normal_images = img2np(os.path.join(data_dir, 'Normal'), select_norm)
glaucoma_images = img2np(os.path.join(data_dir, 'Glaucoma'), select_glau)


In [ ]:

def find_mean_img(full_mat, title, size = (64, 64)):
    # calculate the average
    mean_img = np.mean(full_mat, axis = 0)
    # reshape it back to a matrix
    mean_img = mean_img.reshape(size)
    plt.imshow(mean_img, vmin=0, vmax=255, cmap='Greys_r')
    plt.title(f'Mean {title}')
    plt.axis('off')
    plt.show()
    return mean_img

norm_mean = find_mean_img(normal_images, 'Normal')
glau_mean = find_mean_img(glaucoma_images, 'Glaucoma')

In [ ]:

contrast_mean = norm_mean - glau_mean
plt.imshow(contrast_mean, cmap='bwr')
plt.title(f'Contrast Mean (Normal vs Glaucoma Mean)')
plt.axis('off')
plt.show()

# 2. **Data Preprocessing**



> a) Image Augmentation






In [ ]:
with strategy.scope():
    batch_size = 48
    Augment_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=30,
        width_shift_range=12,
        height_shift_range=12,
        shear_range=0.2,
        zoom_range=[0.8, 1.2],
        horizontal_flip=True,
        rescale=1./255,
        fill_mode='nearest'
    )


In [ ]:
'''
with strategy.scope():
  # Determine the target number of samples to balance classes
  target_num_samples = 141

  # Count the number of samples in each class
  normal_dir = os.path.join(data_dir, 'Normal')
  glaucoma_dir = os.path.join(data_dir, 'Glaucoma')

  # Get the filenames of the original 'normal' and 'glaucoma' images
  normal_filenames = [os.path.join(normal_dir, fname) for fname in os.listdir(normal_dir)]
  glaucoma_filenames = [os.path.join(glaucoma_dir, fname) for fname in os.listdir(glaucoma_dir)]

  num_samples_normal = len(normal_filenames)
  num_samples_glaucoma = len(glaucoma_filenames)

  num_additional_samples_normal = target_num_samples #- num_samples_normal
  num_additional_samples_glaucoma = target_num_samples #- num_samples_glaucoma
'''

In [ ]:
'''
with strategy.scope():

  if num_additional_samples_glaucoma > 0:
      # Load the images from the 'glaucoma' class
      glaucoma_images = [tf.keras.preprocessing.image.load_img(fname, target_size=(256, 256)) for fname in glaucoma_filenames]

      # Convert images to numpy arrays
      glaucoma_images = np.array([tf.keras.preprocessing.image.img_to_array(img) for img in glaucoma_images])

      glau_dir = '/content/drive/MyDrive/Colab Notebooks/Projects/Final(Undergraduate)/[augmented]RIMONE-DL_dataset/Glaucoma'

      # Create a data generator for the 'glaucoma' class with augmentation
      glaucoma_datagen = Augment_generator.flow(glaucoma_images,
                                        batch_size=batch_size,
                                        save_to_dir=glau_dir,
                                        save_prefix='augmented',
                                        save_format='png',
                                        shuffle=True)

      # Generate augmented images to increase the number of samples in the 'glaucoma' class
      num_batches_to_generate_glaucoma = int(np.ceil(num_additional_samples_glaucoma / batch_size))
      remaining_samples_glaucoma = target_num_samples - num_samples_glaucoma

      for _ in range(num_batches_to_generate_glaucoma):
          if remaining_samples_glaucoma > batch_size:
              augmented_images_batch = next(glaucoma_datagen)
              remaining_samples_glaucoma -= batch_size
          else:
              augmented_images_batch = next(glaucoma_datagen)[:remaining_samples_glaucoma]

      # Concatenate the augmented images to the original images
      glaucoma_images = np.concatenate((glaucoma_images, augmented_images_batch), axis=0)
      '''


In [ ]:
'''
with strategy.scope():
  if num_additional_samples_normal > 0:
    # Load the images from the 'normal' class
    normal_images = [tf.keras.preprocessing.image.load_img(fname, target_size=(256, 256)) for fname in normal_filenames]

    # Convert images to numpy arrays
    normal_images = np.array([tf.keras.preprocessing.image.img_to_array(img) for img in normal_images])

    norm_dir = '/content/drive/MyDrive/Colab Notebooks/Projects/Final(Undergraduate)/[augmented]RIMONE-DL_dataset/Normal'

    # Create a data generator for the 'normal' class with augmentation
    normal_datagen = Augment_generator.flow(normal_images,
                                    batch_size=batch_size,
                                    save_to_dir=norm_dir,
                                    save_prefix='augmented',
                                    save_format='png',
                                    shuffle=True)

    # Generate augmented images to increase the number of samples in the 'normal' class
    num_batches_to_generate_normal = int(np.ceil(num_additional_samples_normal / batch_size))
    remaining_samples_normal = target_num_samples - num_samples_normal

    for _ in range(num_batches_to_generate_normal):
        if remaining_samples_normal > batch_size:
            augmented_images_batch = next(normal_datagen)
            remaining_samples_normal -= batch_size
        else:
            augmented_images_batch = next(normal_datagen)[:remaining_samples_normal]

    # Concatenate the augmented images to the original images
    normal_images = np.concatenate((normal_images, augmented_images_batch), axis=0)
'''

In [ ]:
'''
with strategy.scope():
  # Update the total number of images and samples in each class after augmentation
  total_images_normal = normal_images.shape[0]
  total_images_glaucoma = glaucoma_images.shape[0]
  print("Total number of images in 'normal' (after augmentation): ", total_images_normal)
  print("Total number of images in 'glaucoma' (after augmentation): ", total_images_glaucoma)
'''

In [ ]:
from collections import Counter

# Define a custom data generator function that processes images on-the-fly
def custom_data_generator(generator, num_classes):
    for batch_images, batch_labels in generator:
        for i in range(len(batch_images)):
            gray_img = cv2.cvtColor(batch_images[i], cv2.COLOR_BGR2GRAY)
            gray_img = cv2.bilateralFilter(gray_img, 9, 75, 75)

            # Normalize the grayscale image
            normalized_img = cv2.normalize(gray_img, None, 0, 255, cv2.NORM_MINMAX)
            normalized_img = normalized_img.astype('uint8')

            # Apply CLAHE to the normalized and denoised grayscale image
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            clahe_img = clahe.apply(normalized_img)

            # Convert the CLAHE-enhanced image from grayscale to RGB
            clahe_rgb_img = cv2.cvtColor(clahe_img, cv2.COLOR_GRAY2RGB)

            yield clahe_rgb_img, batch_labels[i]



In [ ]:

# Initialize the ImageDataGenerator and train_generator
with strategy.scope():
    dir = '/content/drive/MyDrive/Colab Notebooks/Projects/Final(Undergraduate)/[augmented]RIMONE-DL_dataset'
    generator = ImageDataGenerator(rescale=1./255)
    train_generator = generator.flow_from_directory(
          dir,
          target_size=(256, 256),
          batch_size=batch_size,
          class_mode='categorical'
    )

    num_batches = len(train_generator)
    print('Number of batches: ', num_batches)

    images_per_batch = train_generator.batch_size
    print('Images per batch: ', images_per_batch)

    num_classes = train_generator.num_classes
    print("Number of classes: ", num_classes)



    # Count the number of images in each class
    class_indices = train_generator.classes
    image_counts_per_class = dict(Counter(class_indices))

    print("**Number of images per class**")
    total_images = 0
    for class_index in range(num_classes):
        class_name = list(train_generator.class_indices.keys())[list(train_generator.class_indices.values()).index(class_index)]
        class_count = image_counts_per_class[class_index]
        total_images += class_count
        print(f"Class '{class_name}': {class_count} images")

    print('Total number of images:', total_images)
    num_samples = total_images

    # Plot the distribution of images per class
    plt.bar(image_counts_per_class.keys(), image_counts_per_class.values())
    plt.xticks(range(num_classes), [list(train_generator.class_indices.keys())[i] for i in range(num_classes)], rotation=45)
    plt.xlabel('Class')
    plt.ylabel('Number of Images')
    plt.title('Distribution of Images per Class')
    plt.show()

    # Create a custom data generator
    custom_gen = custom_data_generator(train_generator, num_classes)

    # Process and retrieve the next batch of images and labels
    batch_images, batch_labels = next(custom_gen)


In [ ]:

with strategy.scope():
  # Function to plot three samples from each class
  def plot_samples_per_class(custom_gen, class_names, num_samples=3):
      samples_per_class = {class_name: [] for class_name in class_names}

      # Collect samples from each class
      while any(len(samples) < num_samples for samples in samples_per_class.values()):
          image, label = next(custom_gen)
          class_name = class_names[np.argmax(label)]  # Get the class name

          if len(samples_per_class[class_name]) < num_samples:
              samples_per_class[class_name].append(image)

      # Plot the collected samples
      num_classes = len(class_names)
      fig, axes = plt.subplots(num_samples, num_classes, figsize=(15, 8))

      for class_idx, class_name in enumerate(class_names):
          for sample_idx in range(num_samples):
              if sample_idx < len(samples_per_class[class_name]):
                  axes[sample_idx, class_idx].imshow(samples_per_class[class_name][sample_idx])
                  axes[sample_idx, class_idx].set_title(f"{class_name}")
                  axes[sample_idx, class_idx].axis('off')

      plt.tight_layout()
      plt.show()

  # Get the class names from the generator
  class_names = list(train_generator.class_indices.keys())

  # Call the function to plot three samples from each class
  plot_samples_per_class(custom_gen, class_names)


# 3. **Feature Extraction**

In [ ]:
# Define a function to extract features and labels from images
def extract_features_and_labels(generator, num_batches, num_samples, height, width, channels):
    features = []
    labels = []

    for _ in range(num_batches):
        batch_images, batch_labels = next(generator)
        features.append(batch_images)
        labels.append(batch_labels)

    # Concatenate all batches of images and labels
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)

    # Reshape the features for processing
    features = features.reshape(num_samples, height, width, channels)

    return features, labels

In [ ]:
with strategy.scope():
    height, width = 256, 256
    channels = 3

    # Extract features from the generator
    processed_images, labels = extract_features_and_labels(train_generator, num_batches, num_samples, height, width, channels)

    # Normalize the extracted features
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(processed_images.reshape(-1, height * width * channels))

    normalized_features = normalized_features.reshape(num_samples, height, width, channels)
    print("Shape of normalized features: ", normalized_features.shape)

    # Print the shape of labels
    print("Shape of labels: ", labels.shape)


# 4. **Splitting of data**

In [ ]:
from sklearn.model_selection import KFold

with strategy.scope():
    X = normalized_features
    y = labels

    kf = KFold(n_splits=5, shuffle=True, random_state=23)

    if len(X) >= kf.n_splits:
        for train_index, test_index in kf.split(X):

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # Further split the test set into test and validation sets
            X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=23)
    else:
        print("Not enough samples in the dataset to perform KFold cross-validation.")


    print("X_train shape:", X_train.shape)
    print("X_val shape:", X_val.shape)
    print("X_test shape:", X_test.shape)


# **CNN Model**

In [ ]:
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', name = 'Block1_Conv1', input_shape=input_shape))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', name = 'Block1_Conv2'))
    model.add(MaxPooling2D(pool_size=(2, 2), name = 'Block1_Pool'))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', name = 'Block2_Conv1'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', name = 'Block2_Conv2'))
    model.add(MaxPooling2D(pool_size=(2, 2), name = 'Block2_Pool'))

    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', name = 'Block3_Conv1'))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', name = 'Block3_Conv2'))
    model.add(MaxPooling2D(pool_size=(2, 2), name = 'Block3_Pool'))

    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', name = 'Block4_Conv1'))
    model.add(MaxPooling2D(pool_size=(2, 2), name = 'Block4_Pool'))
    model.add(Dropout(0.25, name = 'Block4_Dropout'))

    model.add(Flatten(name = 'Flatten'))
    model.add(Dropout(0.5, name = 'Dropout'))
    model.add(Dense(units=num_classes, activation='sigmoid', name = 'Output'))
    return model


In [ ]:
with strategy.scope():
  # Defines the input shape
  input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
  cnn_model = create_cnn_model(input_shape)

In [ ]:

with strategy.scope():
  # Compile the model
  cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

  # Model summary
  cnn_model.summary()


In [ ]:

with strategy.scope():
  vis = visualkeras.layered_view(cnn_model, scale_xy=1, scale_z=1, max_z=100, legend = True)
  vis.show()
  # Save the visualization as an image
  plt.savefig('model.png', format='png', bbox_inches='tight', pad_inches=0)

In [ ]:
with strategy.scope():
  # Define the directory to save the model checkpoints
  checkpoint_dir = '/content/drive/MyDrive/Model_Checkpoints/Projects/Glaucoma_Detection'

  # Ensure the directory exists
  os.makedirs(checkpoint_dir, exist_ok=True)

In [ ]:

class ValidationAccuracyStopCallback(Callback):
    def __init__(self, target_accuracy):
        super(ValidationAccuracyStopCallback, self).__init__()
        self.target_accuracy = target_accuracy

    def on_epoch_end(self, epoch, logs=None):
        if logs.get('val_accuracy') >= self.target_accuracy:
            print(f"Stopping training as val_accuracy reached {self.target_accuracy}% or above.")
            self.model.stop_training = True

In [ ]:
with strategy.scope():
  # Define the custom validation accuracy stopping callback
    target_accuracy = 1.0
    val_accuracy_stop = ValidationAccuracyStopCallback(target_accuracy)

In [ ]:
with strategy.scope():
  # Define the EarlyStopping callback
  early_stop = EarlyStopping(monitor='accuracy', patience=10, verbose=1, mode='max')
  epochs = 50

  # Create ModelCheckpoint callback for CNN model
  cnn_checkpoint_path = os.path.join(checkpoint_dir, 'cnn_model_checkpoint.h5')
  cnn_model_checkpoint = ModelCheckpoint(cnn_checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

  # Train the model on the training set
  CNN_history = cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), callbacks = cnn_model_checkpoint)#early_stop, val_accuracy_stop,


# **CAPSULE NETWORK**

In [ ]:
def squash(x, axis=-1, keepdims=True):
    squared_norm = tf.reduce_sum(tf.square(x), axis=axis, keepdims=keepdims)
    scale = squared_norm / (1 + squared_norm) / tf.sqrt(squared_norm + tf.keras.backend.epsilon())
    return scale * x


In [ ]:
def margin_loss(y_true, y_pred):
    margin = 0.5
    margin_positive = K.square(K.relu(1 - margin - y_pred))
    loss = K.sum(y_true * margin_positive, axis=-1)
    return loss


In [ ]:
class Class_Capsule(layers.Layer):
    """
    :param num_capsule: number of capsules in this layer
    :param dim_capsule: dimension of the output vectors of the capsules in this layer
    :param routings: number of iterations for the routing algorithm
    :param share_weights: whether to share weights among capsules
    :param kernel_initializer: initializer for the kernel weights
    :param activation: activation function for the output vectors
    """
    def __init__(self, num_capsule, dim_capsule, routings=3, share_weights=False,
                 kernel_initializer='glorot_uniform', activation='squash', **kwargs):
        super(Class_Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.share_weights = share_weights
        self.kernel_initializer = initializers.get(kernel_initializer)
        if activation == 'squash':
            self.activation = squash
        else:
            self.activation = activations.get(activation)

    def build(self, input_shape):
        assert len(input_shape) >= 3
        input_num_capsule = input_shape[1]
        input_dim_capsule = input_shape[2]
        if self.share_weights:
            self.kernel = self.add_weight(name='capsule_kernel',
                                          shape=(1, input_dim_capsule, self.num_capsule * self.dim_capsule),
                                          initializer=self.kernel_initializer,
                                          trainable=True)
        else:
            self.kernel = self.add_weight(name='capsule_kernel',
                                          shape=(input_num_capsule, input_dim_capsule,
                                                 self.num_capsule * self.dim_capsule),
                                          initializer=self.kernel_initializer,
                                          trainable=True)

    def call(self, inputs, training=None):
        if self.share_weights:
            hat_inputs = K.conv1d(inputs, self.kernel)
        else:
            hat_inputs = K.local_conv1d(inputs, self.kernel, [1], [1])


        batch_size = K.shape(inputs)[0]
        input_num_capsule = K.shape(inputs)[1]
        hat_inputs = K.reshape(hat_inputs,
                               (batch_size, self.num_capsule, input_num_capsule, self.dim_capsule))

        hat_inputs = K.permute_dimensions(hat_inputs, (0, 2, 1, 3))

        b = K.zeros_like(hat_inputs[:, :, 0, :])

        # Begin: Routing algorithm ---------------------------------------------------------------------#
        # # # Simplified routing # # #
        norm_hat_inputs = tf.norm(hat_inputs, axis=-1)
        weighted_hat_inputs = hat_inputs * tf.expand_dims(norm_hat_inputs, axis=-1)
        o = K.sum(weighted_hat_inputs, axis=2) / self.dim_capsule
        o = self.activation(o)
        # End: Routing algorithm -----------------------------------------------------------------------#
        return o

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dim_capsule])

    def get_config(self):
        config = {
            'num_capsule': self.num_capsule,
            'dim_capsule': self.dim_capsule,
            'routings': self.routings
        }
        base_config = super(Class_Capsule, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
def PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
    """
    Apply Conv2D `n_channels` times and concatenate all capsules
    :param inputs: 4D tensor, shape=[None, width, height, channels]
    :param dim_capsule: the dim of the output vector of capsule
    :param n_channels: the number of types of capsules
    :return: output tensor, shape=[None, num_capsule, dim_capsule]
    """
    output = layers.Conv2D(filters=dim_capsule*n_channels, kernel_size=kernel_size, strides=strides, padding=padding,
                           name='primarycap_conv2d')(inputs)
    outputs = layers.Reshape(target_shape=[-1, dim_capsule], name='primarycap_reshape')(output)
    return layers.Lambda(squash, name='primarycap_squash')(outputs)

In [ ]:

with strategy.scope():
    # Get the output of the third convolutional layer
    cnn_output = cnn_model.layers[11].output
    x = Conv2D(filters=128, kernel_size=(7, 7), activation='relu', name='Block5_Conv1')(cnn_output)
    x = Conv2D(filters=128, kernel_size=(6, 6), activation='relu', name='Block5_Conv2')(x)

    # Primary Capsule
    x = PrimaryCap(x, dim_capsule=16, n_channels=2, kernel_size=2, strides=2, padding='valid')

    # Class Capsule Layer
    capsule = Class_Capsule(num_classes, 16, 4, kernel_initializer='glorot_uniform')(x)

    Caps_output = Lambda(lambda z: K.sqrt(K.sum(K.square(z), axis=2)))(capsule)

    CapsNet_model = Model(inputs=cnn_model.input, outputs=Caps_output)

    # Using margin loss
    optimizer = Adam(learning_rate=1e-4) #(0.0001)
    CapsNet_model.compile(loss=margin_loss, optimizer=optimizer, metrics='accuracy')
    CapsNet_model.summary()


In [ ]:
with strategy.scope():
  # Create ModelCheckpoint callback for CapsNet model
  CapsNet_checkpoint_path = os.path.join(checkpoint_dir, 'CapsNet_model_checkpoint.h5')
  CapsNet_model_checkpoint = ModelCheckpoint(CapsNet_checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

  # Train the model on the training set
  CapsNet_history = CapsNet_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), callbacks = CapsNet_model_checkpoint) #early_stop, val_accuracy_stop,


#**GABOR CAPSULE NETWORK**

In [ ]:
class GaborLayer(Layer):
    def __init__(self, filters, kernel_size, strides=(1, 1), activation=None, **kwargs):
        super(GaborLayer, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.activation = activation

    def build(self, input_shape):
        input_channels = input_shape[-1]
        self.kernels_real, self.kernels_imag = self._create_gabor_kernels(input_channels)
        self.built = True

    def call(self, inputs):
        # Perform convolution with complex Gabor filters
        conv_real = K.conv2d(inputs, self.kernels_real, strides=self.strides, padding='valid')
        conv_imag = K.conv2d(inputs, self.kernels_imag, strides=self.strides, padding='valid')

        # Combine real and imaginary parts
        output = K.sqrt(K.square(conv_real) + K.square(conv_imag))

        if self.activation is not None:
            output = self.activation(output)
        return output

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[-1] = self.filters
        return tuple(output_shape)

    def _create_gabor_kernels(self, input_channels):
        kernels_real = []
        kernels_imag = []

        # Generate Gabor filters
        for _ in range(self.filters):
            kernel_real, kernel_imag = self._generate_gabor_filter(self.kernel_size, input_channels)
            kernels_real.append(kernel_real)
            kernels_imag.append(kernel_imag)

        # Stack filters along the channel dimension
        kernels_real = K.stack(kernels_real, axis=-1)
        kernels_imag = K.stack(kernels_imag, axis=-1)

        return kernels_real, kernels_imag

    def _generate_gabor_filter(self, kernel_size, input_channels):
        # Create Gabor filter parameters
        sigma = 0.56 * ((kernel_size[0] - 1) * 0.5 - 1) + 0.25
        theta = np.random.uniform(0, np.pi)
        lambda_ = np.random.uniform(0.5, kernel_size[0] - 0.5)
        psi = np.random.uniform(0, np.pi)
        gamma = np.random.uniform(0.5, 1)

        # Generate Gabor filter
        x0 = (kernel_size[0] - 1) // 2
        y0 = (kernel_size[1] - 1) // 2

        y, x = np.mgrid[-x0:x0 + 1, -y0:y0 + 1]
        distance_from_center = np.sqrt(x**2 + y**2)
        mask = np.where(distance_from_center <= min(x0, y0), 1.0, 0.0)

        x_theta = x * np.cos(theta) + y * np.sin(theta)
        y_theta = -x * np.sin(theta) + y * np.cos(theta)

        kernel_real = mask * np.exp(-0.5 * (x_theta ** 2 + gamma ** 2 * y_theta ** 2) / sigma ** 2) \
                      * np.cos(2 * np.pi * x_theta / lambda_ + psi)
        kernel_imag = mask * np.exp(-0.5 * (x_theta ** 2 + gamma ** 2 * y_theta ** 2) / sigma ** 2) \
                      * np.sin(2 * np.pi * x_theta / lambda_ + psi)

        # Reshape the kernels to match the input channel dimensions
        kernel_real = kernel_real.reshape((kernel_size[0], kernel_size[1], 1))
        kernel_imag = kernel_imag.reshape((kernel_size[0], kernel_size[1], 1))

        # Repeat the kernels for each input channel
        kernel_real = np.repeat(kernel_real, input_channels, axis=-1)
        kernel_imag = np.repeat(kernel_imag, input_channels, axis=-1)

        # Convert the kernels to tensors
        kernel_real = K.constant(kernel_real, dtype=K.floatx())
        kernel_imag = K.constant(kernel_imag, dtype=K.floatx())

        return kernel_real, kernel_imag

    def get_config(self):
        config = super().get_config()
        config.update({
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            "strides": self.strides,
            "activation": self.activation,
         })
        return config


In [ ]:
with strategy.scope():
  # Get the output of the third convolutional layer
  cnn_output = cnn_model.layers[11].output
  x = Conv2D(filters=128, kernel_size=(7, 7), activation='relu', name='Block5_Conv1')(cnn_output)
  print('X: ', x.shape)

  gabor_layer = GaborLayer(filters=128, kernel_size=(5, 5), strides=(1, 1), activation=K.relu, name='GaborLayer')(x)

  #Primary Capsule
  x = PrimaryCap(gabor_layer, dim_capsule=16, n_channels=2, kernel_size=2, strides=2, padding='valid')

  # Secondary Capsule Layer
  capsule = Class_Capsule(num_classes, 16, 4, True)(x)
  Caps_output = Lambda(lambda z: K.sqrt(K.sum(K.square(z), axis=2)))(capsule)

  # Create the combined model
  GCN_model = Model(inputs=cnn_model.input, outputs=Caps_output)

  # Using margin loss
  optimizer = Adam(learning_rate=1e-4) #(0.0001)

  # Compile the model
  GCN_model.compile(loss=margin_loss, optimizer=optimizer, metrics='accuracy')

  # Model summary
  GCN_model.summary() #


In [ ]:

class AccuracyStopCallback(Callback):
    def __init__(self, target_accuracy):
        super(AccuracyStopCallback, self).__init__()
        self.target_accuracy = target_accuracy

    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') >= self.target_accuracy:
            print(f"Stopping training as val_accuracy reached {self.target_accuracy}% or above.")
            self.model.stop_training = True

In [ ]:
with strategy.scope():
  # Define the custom validation accuracy stopping callback
    target_accuracy = 1.0
    accuracy_stop = AccuracyStopCallback(target_accuracy)

In [ ]:
with strategy.scope():
  # Create ModelCheckpoint callback for CapsNet model
  GCN_checkpoint_path = os.path.join(checkpoint_dir, 'GCN_model_checkpoint.h5')
  GCN_model_checkpoint = ModelCheckpoint(GCN_checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

  # Train the model on the training set
  GCN_history = GCN_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=[val_accuracy_stop, GCN_model_checkpoint]) #early_stop, accuracy_stop,


#**Class Activation Maps**

In [ ]:

# Define the function to generate class activation maps using Grad-CAM
def generate_cam(model, img, class_index, layer_name):
    img = np.expand_dims(img, axis=0)
    grad_model = Model(inputs=model.inputs, outputs=(model.get_layer(layer_name).output, model.output))

    with tf.GradientTape() as tape:
        conv_output, predictions = grad_model(img)
        class_output = predictions[:, class_index]

    grads = tape.gradient(class_output, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_output = conv_output[0].numpy()

    for i in range(conv_output.shape[-1]):
        conv_output[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(conv_output, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    return heatmap


In [ ]:

# Convert data_dir to a string
data_dir = str(data_dir)

# Get image filenames for each class
normal_img = [fn for fn in os.listdir(os.path.join(data_dir, 'Normal')) if fn == 'r1_Im037.png']
glaucoma_img = [fn for fn in os.listdir(os.path.join(data_dir, 'Glaucoma')) if fn == 'r2_Im270.png']

# Combine the image filenames for both classes
image_filenames = normal_img + glaucoma_img

# Get class indices and names
class_indices = {'Normal': 0, 'Glaucoma': 1}
class_names = list(class_indices.keys())

# Generate Grad-CAM for one image from each class
for image_filename in image_filenames:
    # Determine the class based on the image filename
    class_name = 'Glaucoma' if image_filename in glaucoma_img else 'Normal'
    class_index = class_indices[class_name]

    # Find the full image path
    image_path = os.path.join(data_dir, class_name, image_filename)
    img = cv2.imread(image_path)
    img = cv2.resize(img, (X_train.shape[1], X_train.shape[2]))
    img = img / 255.0  # Normalize the image

    print('Class index: ', class_index)

    # Generate CAM for CNN model
    cam_cnn = generate_cam(cnn_model, img, class_index, 'Block4_Conv1')

    # Generate CAM for CapsNet model
    cam_capsnet = generate_cam(CapsNet_model, img, class_index, 'Block5_Conv1')

    # Generate CAM for GCN model
    cam_gcn = generate_cam(GCN_model, img, class_index, 'Block5_Conv1')

   # Ensure the heatmap has non-zero values before normalization
    if np.max(cam_cnn) != 0:
        cam_cnn /= np.max(cam_cnn)
    else:
        cam_cnn = np.zeros_like(cam_cnn)


    if np.max(cam_capsnet) != 0:
        cam_capsnet /= np.max(cam_capsnet)
    else:
      cam_capsnet = np.zeros_like(cam_capsnet)

    if np.max(cam_gcn) != 0:
        cam_gcn /= np.max(cam_gcn)
    else:
      cam_gcn = np.zeros_like(cam_gcn)

    # Plot the original image and overlaid CAMs
    plt.figure(figsize=(18, 4))
    plt.subplot(141)
    plt.imshow(img[:, :, ::-1])
    plt.title(f'Original Image - {class_name}')
    plt.axis('off')

    plt.subplot(142)
    plt.imshow(img[:, :, ::-1])
    plt.imshow(cam_cnn, cmap='jet', alpha=0.5)
    plt.title(f'CNN CAM - {class_name}')

    plt.subplot(143)
    plt.imshow(img[:, :, ::-1])
    plt.imshow(cam_capsnet, cmap='jet', alpha=0.5)
    plt.title(f'CapsNet CAM - {class_name}')

    plt.subplot(144)
    plt.imshow(img[:, :, ::-1])
    plt.imshow(cam_gcn, cmap='jet', alpha=0.5)
    plt.title(f'GCN CAM - {class_name}')

    plt.show()



#**EVALUATION**



1. Evaluation on Testing set



In [ ]:
from sklearn.metrics import f1_score, hamming_loss, jaccard_score

# CNN model evaluation
y_pred_cnn = cnn_model.predict(X_test)
y_pred_cnn_binary = (y_pred_cnn > 0.5).astype(int)
cnn_test_loss, cnn_test_acc = cnn_model.evaluate(X_test, y_test, verbose=0)
cnn_test_auc_score = roc_auc_score(y_test, y_pred_cnn)

# CapsNet model evaluation
y_pred_capsnet = CapsNet_model.predict(X_test)
y_pred_capsnet_binary = (y_pred_capsnet > 0.5).astype(int)
capsnet_test_loss, capsnet_test_acc = CapsNet_model.evaluate(X_test, y_test, verbose=0)
capsnet_test_auc_score = roc_auc_score(y_test, y_pred_capsnet)

# GCN model evaluation
y_pred_gcn = GCN_model.predict(X_test)
y_pred_gcn_binary = (y_pred_gcn > 0.5).astype(int)
gcn_test_loss, gcn_test_acc = GCN_model.evaluate(X_test, y_test, verbose=0)
gcn_test_auc_score = roc_auc_score(y_test, y_pred_gcn)


In [ ]:
with strategy.scope():
  # Plotting accuracies
  plt.plot(CNN_history.history['accuracy'], label='Training_CNN', color='blue')
  plt.plot(CapsNet_history.history['accuracy'], label='Training_CapsNet', color='green')
  plt.plot(GCN_history.history['accuracy'], label='Training_GCN', color='red')
  plt.title('Model Accuracies')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend()
  plt.show()


In [ ]:
with strategy.scope():
  # Plotting losses
  plt.plot(CNN_history.history['loss'], label='Training_CNN', color='blue')
  plt.plot(CapsNet_history.history['loss'], label='Training_CapsNet', color='green')
  plt.plot(GCN_history.history['loss'], label='Training_GCN', color='red')
  plt.title('Model Losses')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend()
  plt.show()


#*ROC Curves*

In [ ]:
from sklearn.metrics import roc_curve, auc

# Plotting ROC curve for class 0
fpr, tpr, _ = roc_curve(y_test[:, 0], y_pred_cnn[:, 0])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='CNN', color='blue')

fpr, tpr, _ = roc_curve(y_test[:, 0], y_pred_capsnet[:, 0])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='CapsNet', color='green')

fpr, tpr, _ = roc_curve(y_test[:, 0], y_pred_gcn[:, 0])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='GCN', color='red')

plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()


#*Results*

In [ ]:
# Print evaluation results
print('#----------------------------------------------------------------------------------------#')
print('EVALUATION OF MODELS (TESTING)')
print('#----------------------------------------------------------------------------------------#')

print('1')
print('CONVOLUTIONAL NEURAL NETWORK:')
print("Accuracy:", cnn_test_acc)
print("Loss:", cnn_test_loss)
print('AUC Score: ', cnn_test_auc_score)
print('')

print('2')
print('***CAPSULE NETWORK***')
print("Accuracy:", capsnet_test_acc)
print("Loss:", capsnet_test_loss)
print('AUC Score: ', capsnet_test_auc_score)
print('')

print('3')
print('***GABOR CAPSULE NETWORK***')
print("Accuracy:", gcn_test_acc)
print("Loss:", gcn_test_loss)
print('AUC Score: ', gcn_test_auc_score)
print('')
print('#----------------------------------------------------------------------------------------#')


#*Confusion Matrices*

In [ ]:
with strategy.scope():

    # Get true labels from y_test
    y_true_labels = np.argmax(y_test, axis=1)

    # Convert predictions to labels
    y_pred_cnn_labels = np.argmax(y_pred_cnn, axis=1)
    y_pred_capsnet_labels = np.argmax(y_pred_capsnet, axis=1)
    y_pred_gcn_labels = np.argmax(y_pred_gcn, axis=1)

    # Error Analysis
    for i in range(0, len(y_test)):
        true_label = np.argmax(y_test[i])
        cnn_prediction = np.argmax(y_pred_cnn[i])
        capsnet_prediction = np.argmax(y_pred_capsnet[i])
        gcn_prediction = np.argmax(y_pred_gcn[i])

        if true_label != cnn_prediction or true_label != capsnet_prediction or true_label != gcn_prediction:
            print("True Label:", true_label)
            print("CNN Prediction:", cnn_prediction)
            print("CapsNet Prediction:", capsnet_prediction)
            print("GCN Prediction:", gcn_prediction)
            print("---------------------------------")


In [ ]:
from sklearn.metrics import precision_score, recall_score
# Define a function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, classes, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.2)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=True,  # Set cbar to True
                xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(title)
    plt.show()

In [ ]:

with strategy.scope():
  # Plot confusion matrices
  classes = ["Glaucoma", "Normal"]
  plot_confusion_matrix(y_true_labels, y_pred_cnn_labels, classes, title='CNN Confusion Matrix')
  plot_confusion_matrix(y_true_labels, y_pred_capsnet_labels, classes, title='CapsNet Confusion Matrix')
  plot_confusion_matrix(y_true_labels, y_pred_gcn_labels, classes, title='GCN Confusion Matrix')


In [ ]:
# Calculate precision, recall, and specificity separately
def calculate_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    # Calculate specificity
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)

    return precision, recall, specificity

In [ ]:
with strategy.scope():

    # Calculate precision, recall, and specificity separately
    cnn_precision, cnn_recall, cnn_specificity = calculate_metrics(y_true_labels, y_pred_cnn_labels)
    capsnet_precision, capsnet_recall, capsnet_specificity = calculate_metrics(y_true_labels, y_pred_capsnet_labels)
    gcn_precision, gcn_recall, gcn_specificity = calculate_metrics(y_true_labels, y_pred_gcn_labels)

    # Print precision, recall, and specificity values
    print(f'CNN => Precision: {cnn_precision:.4f}, Recall: {cnn_recall:.4f}, Specificity: {cnn_specificity:.4f}')
    print(f'CapsNet => Precision: {capsnet_precision:.4f}, Recall: {capsnet_recall:.4f}, Specificity: {capsnet_specificity:.4f}')
    print(f'GCN => Precision: {gcn_precision:.4f}, Recall: {gcn_recall:.4f}, Specificity: {gcn_specificity:.4f}')

#**Testing and Inferencing**

In [ ]:
# Directory containing the images for inference
images_directory = '/content/drive/MyDrive/Colab Notebooks/Projects/Final(Undergraduate)/ACRIMA_inference'

# Function to load and preprocess an image
def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.resize((X_train.shape[1], X_train.shape[2]))
    image = np.array(image) / 255.0  # Normalize the image
    return image

# Iterate through all image files in the directory
for filename in os.listdir(images_directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(images_directory, filename)
        test_image = preprocess_image(image_path)

        with strategy.scope():
            # CNN model classification
            cnn_prediction = cnn_model.predict(np.expand_dims(test_image, axis=0))
            cnn_prediction_label = np.argmax(cnn_prediction)
            cnn_prediction_name = class_names[cnn_prediction_label]

            # CapsNet model classification
            capsnet_prediction = CapsNet_model.predict(np.expand_dims(test_image, axis=0))
            capsnet_prediction_label = np.argmax(capsnet_prediction)
            capsnet_prediction_name = class_names[capsnet_prediction_label]

            # GCN model classification
            gcn_prediction = GCN_model.predict(np.expand_dims(test_image, axis=0))
            gcn_prediction_label = np.argmax(gcn_prediction)
            gcn_prediction_name = class_names[gcn_prediction_label]

        # Display the image
        plt.imshow(test_image)
        plt.axis('off')
        plt.show()

        # Print classifications
        print("Image:", filename)
        print("CNN Model Classification:", cnn_prediction_name)
        print("CapsNet Model Classification:", capsnet_prediction_name)
        print("GCN Model Classification:", gcn_prediction_name)


# **Saving and Converting best model**

In [ ]:
with strategy.scope():
    drive.mount('/content/drive')

    drive_path = '/content/drive/MyDrive/Colab Notebooks/Projects/Final(Undergraduate)/RIMONE-DL_dataset'
    file_name = 'best_model.h5'
    file_path = os.path.join(drive_path, file_name)

    if os.path.isfile(file_path):
        print('File exists:', file_path)
    else:
        print('File does not exist:', file_path)


In [ ]:
from tensorflow.keras.models import save_model

with strategy.scope():
    # Specify the path to save the best model
    save_path = '/content/drive/MyDrive/Colab Notebooks/Projects/Final(Undergraduate)/Saved_Models/model.h5'

    # Determine the best model based on a chosen metric (e.g., accuracy and loss)
    best_model_name = None
    best_accuracy = 0.0
    best_loss = float('inf')

    # Check if the CNN model has the best metric
    if cnn_test_acc > best_accuracy and cnn_test_loss < best_loss:
        best_accuracy = cnn_test_acc
        best_loss = cnn_test_loss
        best_model_name = 'CNN model'
        save_model(cnn_model, save_path)  # Save the best CNN model

    # Check if the CapsNet model has the best metric
    if capsnet_test_acc > best_accuracy and capsnet_test_loss < best_loss:
        best_accuracy = capsnet_test_acc
        best_loss = capsnet_test_loss
        best_model_name = 'CapsNet model'
        save_model(CapsNet_model, save_path)  # Save the best CapsNet model

    # Check if the GCN model has the best metric
    if gcn_test_acc > best_accuracy and gcn_test_loss < best_loss:
        best_accuracy = gcn_test_acc
        best_loss = gcn_test_loss
        best_model_name = 'Gabor CapsNet model'
        save_model(GCN_model, save_path)  # Save the best GCN model

    # Define the path for the labels file
    labels_file_path = '/content/drive/MyDrive/Colab Notebooks/Projects/Final(Undergraduate)/TfLite/labels.txt'

    # Save the labels and class names as a .txt file
    with open(labels_file_path, 'w') as f:
        for class_name, label_idx in train_generator.class_indices.items():
            f.write(class_name + '(' + str(label_idx) + ')' + '\n')

    print('Labels and class names saved as .txt:', labels_file_path)

# Print the details of the best model
print('Best Model:')
print('Accuracy:', best_accuracy)
print('Loss:', best_loss)
print('Model Name:', best_model_name)


In [ ]:
# Path to the saved model in .h5 format
saved_model_path = '/content/drive/MyDrive/Colab Notebooks/Projects/Final(Undergraduate)/Saved_Models/model.h5'

# Define the custom objects dictionary for loading custom layers
custom_objects = {'Class_Capsule': Class_Capsule, 'margin_loss': margin_loss, 'squash': squash}

# Load the saved model with custom layers and loss function
model = tf.keras.models.load_model(saved_model_path, custom_objects=custom_objects, compile=False)

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
tflite_model_path = '/content/drive/MyDrive/Colab Notebooks/Projects/Final(Undergraduate)/TfLite/model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print('Model converted and saved as TensorFlow Lite:', tflite_model_path)
